In [23]:
%load_ext rpy2.ipython
import warnings
warnings.filterwarnings('ignore')

### Download and decompress raw rMATS-turbo output file for CCLE dataset
- **<font color='red'>This step is optional and is omitted by default</font>** since in the plot step, there will be an example dataset (with a smaller subset of events) for illustration purpose. Users can remove the `%%script false` in the following cell to enable it.
- **<font color='red'>Note that the downloading step takes longer to execute.</font>** It is better to run it using command line.

In [12]:
%%script false
%%bash
mkdir -p ./CCLE/rmats
cd ./CCLE/rmats
wget https://zenodo.org/record/6647024/files/CCLE.tar.gz  # This steps takes a long time.
tar -zxvf CCLE.tar.gz
mv CCLE post_1019

### Filtering of rMATS-turbo result
- **<font color='red'>This step is optional and is omitted by default</font>** since in the plot step, there will be an example dataset (with a smaller subset of events) for illustration purpose. Users can remove the `%%script false` in the following cell to enable it.
- **<font color='red'>Note that this step will take long to finish since the input file contains a huge number of events.</font>** It is better to run it outside of Jupyter Notebook.
- folders contain the input file: ./CCLE/rmats/post_1019/
- folders contain the output file: ./CCLE/rmats_filtering/

In [16]:
%%script false
import os, sys
import numpy as np
import math

sys.path.append(os.path.abspath("./scripts"))
from class_exon import get_exon_class

def read_rMATS(fn, readCov, minPSI, maxPSI, sigFDR, sigDeltaPSI, bgFDR, bgWithinGroupDeltaPSI):
    exon, event_type = get_exon_class(fn)
    filtered_event_list = []
    up_event_list = []
    dn_event_list = []
    bg_event_list = []
    with open(fn, "r") as f:
        header = f.readline()
        for line in f:
            x = exon(line)
            if (
                x.averageCountSample1 >= readCov
                and x.averageCountSample2 >= readCov
                and np.nanquantile(x.IncLevel1 + x.IncLevel2, 0.05) <= maxPSI
                and np.nanquantile(x.IncLevel1 + x.IncLevel2, 0.95) >= minPSI
                and len(x.chrom) <= 5
            ):
                filtered_event_list.append(x)
                if x.FDR <= sigFDR:
                    if x.IncLevelDifference >= sigDeltaPSI:
                        dn_event_list.append(x)
                    elif x.IncLevelDifference <= -sigDeltaPSI:
                        up_event_list.append(x)
                elif (
                    x.FDR >= bgFDR
                    and max(x.IncLevel1) - min(x.IncLevel1) <= bgWithinGroupDeltaPSI
                    and max(x.IncLevel2) - min(x.IncLevel2) <= bgWithinGroupDeltaPSI
                ):
                    bg_event_list.append(x)
            # considering cases when --b2 is not provided.
            elif (
                x.averageCountSample1 >= readCov
                and math.isnan(x.averageCountSample2)
                and np.nanquantile(x.IncLevel1 + x.IncLevel2, 0.05) <= maxPSI
                and np.nanquantile(x.IncLevel1 + x.IncLevel2, 0.95) >= minPSI
                and len(x.chrom) <= 5
            ):
                filtered_event_list.append(x)
    event_dict = {
        "upregulated": up_event_list,
        "downregulated": dn_event_list,
        "background": bg_event_list,
        "filtered": filtered_event_list,
    }
    return header, event_dict


if __name__ == "__main__":
    fn_rmats = './CCLE/rmats/post_1019/SE.MATS.JC.txt'
    dir_output = './CCLE/rmats_filtering'
    if not os.path.exists(dir_output):
        os.makedirs(dir_output)
    fn_up, fn_dn, fn_bg, fn_filtered = [os.path.join(dir_output, i + os.path.basename(fn_rmats)) 
                                        for i in ["up_", "dn_", "bg_", "filtered_"]]

    #### FILTERING PARAMETERS ####
    # defalt filtering parameter
    readCov = 20
    minPSI = 0.05
    maxPSI = 0.95
    sigFDR = 0.01
    sigDeltaPSI = 0.05
    bgFDR = 0.5
    bgWithinGroupDeltaPSI = 0.2
    
    #### RUN ####
    header, event_dict = read_rMATS(
        fn_rmats, readCov, minPSI, maxPSI, sigFDR, sigDeltaPSI, bgFDR, bgWithinGroupDeltaPSI
    )
    with open(fn_filtered, "w") as f:
        f.write(header)
        for event in event_dict["filtered"]:
            f.write(str(event))
    if len(event_dict["upregulated"]) > 0 or len(event_dict["downregulated"]) > 0 or len(event_dict["background"]) > 0:
        with open(fn_up, "w") as f:
            f.write(header)
            for event in event_dict["upregulated"]:
                f.write(str(event))
        with open(fn_dn, "w") as f:
            f.write(header)
            for event in event_dict["downregulated"]:
                f.write(str(event))
        with open(fn_bg, "w") as f:
            f.write(header)
            for event in event_dict["background"]:
                f.write(str(event))


### Extract PSI value from rMATS-turbo result
- **<font color='red'>This step is optional and is omitted by default</font>** since in the plot step, there will be an example dataset (with a smaller subset of events) for illustration purpose. Users can remove the `%%script false` in the following cell to enable it.
- **<font color='red'>Note that this step will take long to finish since the input file contains a huge number of events.</font>** It is better to run it using command line.
- Input file: 
    - `./CCLE/rmats/post_1019/SE.MATS.JC.txt`
- Output file: 
    - `./CCLE/rmats/psi/SE.MATS.JC_COUNT.txt`
    - `./CCLE/rmats/psi/SE.MATS.JC_PSI.txt`
    - `./CCLE/rmats_filtering/filtered_SE.MATS.JC_PSI.txt`

In [21]:
%%script false
%%bash
sc_PSI_count=./scripts/extract_PSI_count.py
declare -a event_array=("SE")     # only look at SE events. Other available option: 'RI', 'MXE', 'A5SS', 'A3SS'
declare -a counttype_array=("JC") # only look at JC count type. Other available option: 'JCEC'

mkdir -p ./CCLE/rmats/psi
for event in "${event_array[@]}"; do
    for counttype in "${counttype_array[@]}"; do
        python $sc_PSI_count ./CCLE/rmats/post_1019/${event}.MATS.${counttype}.txt ./CCLE/rmats/psi
    done
done

######## insert sample IDs #########
metadata=./CCLE/metadata/metadata.txt
samples=$(awk -F '\t' '{print $1}' ${metadata} | tr '\n' '\t' | sed 's/\t$/\n/')
sed -i "1i ID\t${samples}" ./CCLE/rmats/psi/*_COUNT.txt
sed -i "1i ID\t${samples}" ./CCLE/rmats/psi/*_PSI.txt

awk -F '\t' 'FNR==NR{a[$1]=$1;next} ($1 in a) {print $0}' \
    ./CCLE/rmats_filtering/filtered_SE.MATS.JC.txt \
    ./CCLE/rmats/psi/SE.MATS.JC_PSI.txt \
    > ./CCLE/rmats_filtering/filtered_SE.MATS.JC_PSI.txt


### Prep: EMT score calculation
- **<font color='red'>This step is optional</font>** since the output matrix `./CCLE/plot_heatmap/EMT_score_CCLE.txt` has already been provided in the GitHub repository.
- Input files:
    - `./CCLE/plot_heatmap/EMT_signature_genes_in_gencode.v31lift37.txt`: file contains 207 EMT signature genes
    - `./CCLE/exp/exp_FPKM_EMT_signature_genes_in_gencode.v31lift37.txt`: file contains the expression value of 207 signature genes for all samples. Dimention genes x samples
- Output matrix:
    - output matrix would be a matrix with three columns (STATISTIC, pval, fdr)
        - where STATISTIC represent the KS Statistics, -1 -- more epithelial; 1 -- more mysenchymal
    - the matrix will be stored in `./CCLE/plot_heatmap/EMT_score_CCLE.txt`.

In [2]:
%%R
library(zeallot)
KS_test <- function(x, y) {
    n.x <- length(x)
    n.y <- length(y)

    w <- c(x, y)

    z <- cumsum(ifelse(order(w) <= n.x, 1/n.x, -1/n.y))
    z <- z[c(which(diff(sort(w)) != 0), n.x + n.y)] #exclude ties

    alternative <- "two.sided"
    STATISTIC <- switch(alternative, two.sided = z[which.max(abs(z))], 
                        greater = max(z), less = -min(z))

    test = ks.test(x, y)
    return(c(STATISTIC, test$p.value))
}

exp2EMT <- function(exp, signature) {
    # exp is a matrix of expression value; colname is sample name
    # signature is a vector of EMT signature corresponding to the rows of exp matrix
    result = data.frame(STATISTIC = NULL, pval = NULL)
    for (sample in colnames(exp)) {
        exp_E <- exp[which(signature == 'Epi'), sample]
        exp_M <- exp[which(signature == 'Mes'), sample]
        c(STATISTIC, pval) %<-% KS_test(exp_E, exp_M)
        result = rbind(result, data.frame(STATISTIC= STATISTIC, pval = pval))
    }
    rownames(result) = colnames(exp)
    return(result)
}

In [3]:
%%R
exp <- read.table('./CCLE/exp/exp_FPKM_EMT_signature_genes_in_gencode.v31lift37.txt', header = T, sep = '\t', quote = '', stringsAsFactors = FALSE, row.names = 1)
EMT_gene <- read.table('./CCLE/plot_heatmap/EMT_signature_genes_in_gencode.v31lift37.txt', header = T, sep = '\t', quote = '', stringsAsFactors = FALSE, row.names = 7)
result = exp2EMT(exp[, -c(1)], EMT_gene[rownames(exp), 'EM_Signature'])
result$fdr = p.adjust(result$pval)
write.table(result, './CCLE/plot_heatmap/EMT_score_CCLE.txt', row.names = T, col.names = T, quote = FALSE, sep = '\t')

### Plot Heatmap
- Input files
    - `./CCLE/plot_heatmap/EMT_score_CCLE.txt`: EMT score matrix generated by running the code described earlier
    - `./CCLE/plot_heatmap/EMT_EMBO.txt`: EMT score retreived from reference paper (EMBO)
    - `./CCLE/plot_heatmap/EMT_NBT.txt`: EMT classification retreived from reference paper (NBT)
    - `./CCLE/plot_heatmap/example_subset_SE.MATS.JC_PSI.txt`: PSI value extracted from rMATS-turbo output; using a subset of events for illustration purpose. Users can also get the PSI value for the full list of events `./CCLE/rmats_filtering/filtered_SE.MATS.JC_PSI.txt` by running the code descriped ealier:
        - Download and decompress raw rMATS-turbo output file for CCLE dataset
        - Filtering of rMATS-turbo result
        - Extract PSI value from rMATS-turbo result

#### calculate correlation between PSI value and EMT score
- **<font color='red'>This step is optional</font>** since the output matrix `./CCLE/plot_heatmap/CCLE_correlation_EMT-filteredPSI.txt` has already been provided in the GitHub repository.
- Output:
    - `./CCLE/plot_heatmap/CCLE_correlation_EMT-filteredPSI.txt`: correlation between EMT score and PSI value
- Note:
    - This step will take several minutes to process if the input file contains lots of events (e.g. ~8min for 50,000 events)

In [24]:
%%R
EMT <- read.table('./CCLE/plot_heatmap/EMT_score_CCLE.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F)
PSI <- read.table('./CCLE/plot_heatmap/example_subset_SE.MATS.JC_PSI.txt', header = T, sep = '\t', row.names = 1, quote = "", stringsAsFactor = F)

result = list()
for (ID in rownames(PSI)){
    mydf = data.frame(EMT = EMT$STATISTIC, PSI = as.numeric(PSI[ID, rownames(EMT)]))
    fit <- lm(EMT ~ PSI, data = mydf)
    fit_summary = summary(fit)
    result[[ID]] = setNames(c(fit_summary$r.squared, fit$coefficient[2], fit$coefficient[1]), 
                            c('r.squared', 'coefficient', 'intercept'))
}
myresult = do.call(rbind, result)

write.table(myresult, './CCLE/plot_heatmap/CCLE_correlation_EMT-filteredPSI.txt', sep = '\t', quote = FALSE, row.names = T, col.names = T)


#### Filter out events whose PSI value is highly correlated with EMT score (R2 > 0.4)
- Output
    - `./CCLE/plot_heatmap/r2_0.4_SE.MATS.JC_PSI.txt`: PSI values of events highly correlated with EMT scores

In [5]:
%%bash
cd ./CCLE/plot_heatmap
awk -F '\t' '($2 > 0.4) {print $0}' CCLE_correlation_EMT-filteredPSI.txt > r2_0.4_CCLE_correlation_EMT-filteredPSI.txt
sed -i '1s/^/ID\t/' r2_0.4_CCLE_correlation_EMT-filteredPSI.txt
awk -F '\t' 'FNR==NR{a[$1]=$1;next} ($1 in a) {print $0}' r2_0.4_CCLE_correlation_EMT-filteredPSI.txt example_subset_SE.MATS.JC_PSI.txt > r2_0.4_SE.MATS.JC_PSI.txt
rm r2_0.4_CCLE_correlation_EMT-filteredPSI.txt

#### Plot

In [6]:
%%R
library(circlize)
library(ComplexHeatmap)
set.seed(1993)

# EMT score and cell line annotation metadata
EMT_score <- read.table('./CCLE/plot_heatmap/EMT_score_CCLE.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F)
metadata <- read.table('./CCLE/metadata/metadata.txt', header = T, sep = '\t', stringsAsFactor = FALSE, quote = "", check.name = F, row.names = 1)
EMT <- cbind(EMT_score, metadata[rownames(EMT_score), ])
EMT <- EMT[with(EMT, order(Tissue, STATISTIC)),]

# from EMBO paper
EMT_score_EMBO <- read.table('./CCLE/plot_heatmap/EMT_EMBO.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F, row.names = 1)
EMT_score_EMBO = EMT_score_EMBO[rownames(EMT), ]
EMT_EMBO = apply(EMT_score_EMBO, 1, 
      FUN = function(x){
          score = as.numeric(x['Generic.EMT.CL.Ksscore'])
          pval = as.numeric(x['Generic.EMT.CL.pv'])
          if (is.na(score)) {
              EM = NA
          } else if (score < 0) {
              EM = ifelse(pval < 0.05, 'Epi', 'intermediate Epi')
          } else {
              EM = ifelse(pval < 0.05, 'Mes', 'intermediate Mes')
          }
          return(EM)
      })


# from NBT paper
EMT_NBT <- read.table('./CCLE/plot_heatmap/EMT_NBT.txt', header = T, sep = '\t', stringsAsFactor = F, quote = "", check.name = F, row.names = 1)
EMT_NBT <- EMT_NBT[rownames(EMT), ]
EMT_NBT[which(EMT_NBT$EMT_status == 'Epithelial'), 'EMT_status'] = 'Epi'
EMT_NBT[which(EMT_NBT$EMT_status == 'Mesenchymal'), 'EMT_status'] = 'Mes'
EMT_NBT[which(EMT_NBT$EMT_status == 'OtherEMT'), 'EMT_status'] = 'Other'

################ ht_CCLE ################
mydf <- read.table('./CCLE/plot_heatmap/r2_0.4_SE.MATS.JC_PSI.txt', header = T, sep = '\t', row.names = 1, quote = "", stringsAsFactor = F)
mydf_CCLE <- mydf[, rownames(EMT)]
mat <- t(apply(mydf_CCLE, 1, scale))


column_ha = HeatmapAnnotation(#'EMT score' = anno1,
                              'EMT score' = anno_points(EMT$STATISTIC, 
                                                        ylim = c(-1, 1), 
                                                        size = unit(0.8, "mm"),
                                                        gp = gpar(col = ifelse(EMT$STATISTIC > 0 & EMT$pval < 0.05, "dodgerblue4", ifelse(EMT$STATISTIC < -0 & EMT$pval < 0.05, "orangered4", "grey"))),
                                                        axis_param = list(side = "left", at = c(-1, 0, 1), labels = c("Epi -1", "0", "Mes  1"))
                                                         ), 
                              'Tan et al., 2014' = factor(EMT_EMBO, levels = c('Epi', 'intermediate Epi', 'intermediate Mes', 'Mes', 'NA')), 
                              'Klijn et al., 2015' = factor(EMT_NBT$EMT_status, levels = c('Epi', 'Other', 'Mes', 'NA')),
                              'disease stage' = factor(EMT$disease_stage, levels = c('benign_neoplasia', 'primary', 'metastasis')),
                              'tissue' = EMT$Tissue,
                              col = list('disease stage' = c("benign_neoplasia" = "darkgrey", "primary" = "turquoise", "metastasis" = "deeppink"),
                                         'Tan et al., 2014' = c('Epi' = 'orangered4', 'intermediate Epi' = 'orangered', 'intermediate Mes' = 'dodgerblue', 'Mes' = 'dodgerblue4', 'NA' = 'white'), 
                                         'Klijn et al., 2015' = c('Epi' = 'orangered4', 'Other' = 'grey', 'Mes' = 'dodgerblue4', 'NA' = 'white')), 
                              na_col = "white")
lgd_EMT = Legend(labels = c('Epi', 'intermediate', 'Mes'), legend_gp = gpar(fill = c('orangered4', 'grey', 'dodgerblue4')), title = "EMT score")


ht_CCLE  = Heatmap(mat, 
            show_row_names = FALSE,
            na_col = "gray30",
            column_split = EMT$Tissue,
            top_annotation = column_ha,
            # right_annotation = row_ha,
            cluster_columns = F, 
            heatmap_legend_param = list(
                title = "CCLE PSI Z-Score",
                legend_height = unit(4, "cm"),
                #direction = "horizontal",
                title_position = "leftcenter-rot"),
            column_title = 'Cancer Cell Line Encyclopedia (CCLE)'
        )

################ PLOT ################
pdf("./CCLE/plot_heatmap/plot_heatmap_sortedByEMT_usingCorrelation.pdf", width = 15, height = 6)
lgd_list = list('EMT score' = lgd_EMT)

ht = draw(
    ht_CCLE,
    annotation_legend_list = lgd_list,
    annotation_legend_side = "right",
    heatmap_legend_side = "right"
)

dev.off()

png 
  2 
